In [126]:
import numpy as np

In [127]:
def evaluate(x):
    '''Evaluate our objective function'''
    px = np.array([0,  50, 100, 300, 400, 700, 799], dtype=int)
    py = np.array([0, 100,   0,   0,  25,   0,  50], dtype=int)
    xx = np.arange(0, 800)
    yy = np.interp(xx, px, py)
    return -yy[x]  # negative altitude, becase we are minimizing (to be consistent with other obj. functions)

In [128]:
def update_dist(best):
    """
    Update the empirical distribution according to 
    the known value in several points
    """
    
    x = np.arange(800)
    dist = np.interp(x, x[best<0], np.abs(best[best<0]))
    # normalize distribution
    dist = (1/np.sum(dist))*dist
    return dist

In [129]:
def sample_dist(dist, per_value_samples):
    """
    Generate random samples from our current knowledge about 
    the objective function, represented by empirical 
    distribution 'dist'
    """

    #ratios = np.round(10000*dist)
    rng = np.random.rand(per_value_samples,800)
    large_dist = np.array([dist,]*per_value_samples)
    large_x = np.array([np.arange(800),]*per_value_samples)

    pick = rng < large_dist
    sample = large_x[pick].reshape(np.sum(pick))
    shuffle = np.random.permutation(np.sum(pick))

    return sample[shuffle[:10]]

In [130]:
def eda(mcarlo_samples):
    """ 
    A very simplified and naive implementation of 
    Estimation of Distribution (EDA) optimization algorithm
    """

    x = np.random.randint(0, 800, size=(11,10))
    y = np.zeros(shape=(10,10))
    dist = (1/800)*np.ones(shape=(10,800))
    best = np.zeros(800)
    last_change = np.zeros(800)
    for i in range(10):
        y[i,:] = evaluate(x[i,:])
        ind = [k for k in x[i,:]]
        pos = y[i,:] < best[x[i,:]]
        last_change[x[i,pos]] = i*np.ones(1,np.sum(pos))
        best[x[i,:]] = np.minimum(best[x[i,:]], y[i,:])

        dist[i,:] = update_dist(best)
        x[i+1,:] = sample_dist(dist[i,:], mcarlo_samples)
    final_x = np.argmin(best)
    final_y = np.min(best)
    return final_y, last_change[final_x]

In [131]:
def run_multiple(n, fun):
    """
    Evaluate function 'fun' 'n' times and measure times it 
    reached the target value, average score and 
    number of iterations needed
    """

    best_val = np.zeros(n)
    iter_no = np.zeros(n)
    for i in range(n):
        best_val[i], iter_no[i] = fun()
    
    target_reached = np.sum(best_val == -100)
    average_score = np.mean(best_val)
    average_iters = np.mean(iter_no)

    # since we begin with zero and each run is 10 evaluations
    average_iters = 10*(average_iters + 1)

    print(f"Targer value -100 reached in {target_reached} runs, which is {target_reached*100/1000} %.")
    print(f"Average score is {average_score}.")
    print(f"Average number of iterations to converge is {average_iters}.")

    return target_reached, average_score, average_iters

In [141]:
def g():
    return eda(70)
ex_target, ex_score, ex_iters = run_multiple(1000, g)

Targer value -100 reached in 395 runs, which is 39.5 %.
Average score is -96.92519444444446.
Average number of iterations to converge is 63.879999999999995.
